In [3]:
import sys
import os

In [4]:
wd = os.getcwd()
sys.path.append(f'{wd}/../modules')

In [5]:
root_input_folder    = f'{wd}/../input_files'
root_output_folder   = f'{wd}/../output_files'

# Functions

In [39]:
def write_lj_fluid_sw_interaction_file(args, input_folder, output_folder, label):
    # Unpack arguments
    Ns, Nw, Lx, Ly, Lz, epsilon, sigma, eta, system_temp, pressure, time_step, n_step_to_run, m_frame_dump, m_frame_restart = args

    # System-specific constants
    kappa = 1000  # Spring constant for wall-wall interactions, in epsilon*sigma^-2
    r0 = 1.1626  # Equilibrium distance for wall-wall interactions, in sigma

    script = f"""# LJ Fluid with Walls and Harmonic Wall Interactions
units        lj
atom_style   full
dimension    3
boundary     p p p

# System variables
variable     label      equal   {label}
variable     Ns         equal   {Ns}              # Number of solvent particles
variable     Nw         equal   {Nw}              # Number of wall particles
variable     Lx         equal   {Lx}              # Box x length
variable     Ly         equal   {Ly}              # Box y length
variable     Lz         equal   {Lz}              # Box z length, adjustable based on pressure
variable     epsilon    equal   {epsilon}         # Interaction energy scale
variable     sigma      equal   {sigma}           # Particle size
variable     eta        equal   {eta}             # Interaction strength parameter
variable     system_temp equal  {system_temp}     # System temperature
variable     pressure   equal   {pressure}        # System pressure
variable     dt         equal   {time_step}       # Time step
variable     kappa      equal   {kappa}           # Spring constant for wall-wall harmonic interactions
variable     r0         equal   {r0}              # Equilibrium distance for wall-wall interactions

# Create simulation box
region       box block 0 {Lx} 0 {Ly} 0 {Lz}
create_box   2 box

# Pair styles for non-bonded interactions
pair_style   hybrid/overlay lj/cut {2.5*sigma}
pair_coeff   1 1 lj/cut {epsilon} {sigma} {2.5*sigma}  # s-s interactions
pair_coeff   1 2 lj/cut {epsilon} {sigma} {pow(2, 1/6)*sigma}  # s-w interactions using WCA potential

# Create particles
create_atoms 1 random {Ns} 12345 box
create_atoms 2 random {Nw} 67890 box

# Set masses
mass 1 1.0
mass 2 1.0

print "Particles created"

# Initial equilibration with harmonic interactions among wall particles
bond_style   harmonic
bond_coeff   2 {kappa} {r0}

print "Bound spring created"

# NOTE: Define bonds between wall particles here, assuming a pre-defined list or using a script to generate

timesetp {time_step}

# Initial equilibration in NPT ensemble
fix          1 all npt temp {system_temp} {system_temp} (10*${{dt}}) aniso {pressure} {pressure} (100*${{dt}})
thermo       100
run          5000  # Duration for initial equilibration

# For production runs, freeze wall particles in their final equilibrium positions
unfix        1  # Remove NPT ensemble fix
fix          2 wall setforce 0.0 0.0 0.0  # Freeze wall particles

# Continue with production run
run          {n_step_to_run}

write_restart {output_folder}/${{label}}.restart
"""

    # Write to file
    with open(f'{output_folder}/{label}.in', 'w') as file:
        file.write(script)


# Define experiment

In [40]:
import numpy as np
import pandas as pd
from spartian_tools import *

In [41]:
stage   = 'BUILD'
label   = 'build_lj'

length_x = 10.0
length_y = 10.0
length_z = 10.0

time_step   = 0.001
system_temp = 2.0

n_step_to_run   = 2000000
m_frame_dump    = 300
m_frame_restart = 20000

pressure = 2.65
epsilon = 1
sigma = 1
eta = 1

Ns = 1000
Nw = 2000

In [42]:
experiment_label    = f'{stage}_TEST'
output_folder       = f'{root_output_folder}/{stage}/{experiment_label}'

create_experiment_folder(output_folder)

args = tuple([Ns, Nw, length_x, length_y, length_z, epsilon, sigma, eta, system_temp, pressure, time_step, n_step_to_run, m_frame_dump, m_frame_restart ])
write_lj_fluid_sw_interaction_file(args, root_input_folder, output_folder, label)

submit_file = f'{output_folder}/{label}.submit'
write_submit_jobs_SLURM(label, submit_file)
#submit_jobs_SLURM(output_folder, submit_file)